In [1]:
import torch
from peft import (
    PeftConfig,
    PeftModel
)
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer
)

import pandas as pd

from collections import Counter

In [2]:
# load the test data
df = pd.read_csv('../data/test_person_10.csv')

In [3]:
PEFT_MODEL = "./models/llama-2-7b-chat-hf-llm-emo-person-10-finetuned-peft/"

config = PeftConfig.from_pretrained(PEFT_MODEL)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    device_map="auto",
    trust_remote_code=True
)

tokenizer=AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

model = PeftModel.from_pretrained(model, PEFT_MODEL)

# %%
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

# if cuda is available, use it
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# prompt = """
# <human>: midjourney prompt for a boy running in the snow
# <context>: 
# <assistant>:
# """.strip()

# encoding = tokenizer(prompt, return_tensors="pt").to(device)
# with torch.inference_mode():
#   outputs = model.generate(
#       input_ids = encoding.input_ids,
#       attention_mask = encoding.attention_mask,
#       generation_config = generation_config
#   )

# print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

In [4]:
# make a list with the unique values of ExperiencedEmotion1 column
emotions = df['ExperiencedEmotion1'].unique()

# make a copy of the dataframe
df_copy = df.copy()

# iterate over the dataset, User column is <human> and <context> is the context and <assistant> is the model response
for index, row in df.iterrows():
    prompt = f"""
    <human>: {row['User']}
    <context>: {row['context']}
    <assistant>:
    """.strip()

    encoding = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.inference_mode():
        outputs = model.generate(
            input_ids = encoding.input_ids,
            attention_mask = encoding.attention_mask,
            generation_config = generation_config
        )

    # Decode the assistant's response
    assistant_response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract the part of the assistant's response that comes after '<assistant>'
    assistant_response = assistant_response.split('<assistant>')[-1].strip().lower()

    # Check if the assistant's response contains a valid emotion
    emotion_counter = Counter()
    for emotion in emotions:
        emotion_counter[emotion] = assistant_response.count(emotion.lower())

    if emotion_counter:
        # If a valid emotion is found and it's the same as the ground truth, keep the ground truth in df_copy
        most_common_emotion = emotion_counter.most_common(1)[0][0]
        if row['ExperiencedEmotion1'].lower() in [emotion for emotion, count in emotion_counter.items() if count > 0]:
            df_copy.loc[index, 'ExperiencedEmotion1'] = row['ExperiencedEmotion1']
        else:
            # If a valid emotion is found but it's not the same as the ground truth, replace the ground truth with the most common emotion in df_copy
            df_copy.loc[index, 'ExperiencedEmotion1'] = most_common_emotion
    else:
        # If no valid emotion is found, put 'REST' in df_copy
        df_copy.loc[index, 'ExperiencedEmotion1'] = 'REST'

KeyboardInterrupt: 

In [ ]:
# drop all the columns except for the 'ExperiencedEmotion1' column
df_copy = df_copy['ExperiencedEmotion1']

# save the df_copy to a csv file as prediciton_peroosn_10.csv
df_copy.to_csv('../data/prediction_person_10.csv', index=False)